In [1]:
def greedy_shrink(ls, constraint, shrink):
    while True:
        for s in shrink(ls):
            if constraint(s):
                ls = s
                break
        else:
            return ls

In [2]:
def shrink1(ls):
    for i in range(len(ls)):
        s = list(ls)
        if s[i] > 0:
            s[i] -= 1
            yield list(s)
        del s[i]
        yield list(s)

In [3]:
def show_trace(start, constraint, simplifier):
    if start is None:
        while True:
            start = gen_list()
            if constraint(start):
                break

    shrinks = [0]
    tests = [0]

    def print_shrink(ls):
        tests[0] += 1
        if constraint(ls):
            shrinks[0] += 1
            print("✓", ls)
            return True
        else:
            print("✗", ls)
        return False
    print("✓", start)
    simplifier(start, print_shrink)
    print()
    print("%d shrinks with %d function calls" % (
            shrinks[0], tests[0]))

In [4]:
from functools import partial

In [5]:
show_trace([5, 5], lambda x: len(x) >= 2, partial(greedy_shrink, shrink=shrink1))

✓ [5, 5]
✓ [4, 5]
✓ [3, 5]
✓ [2, 5]
✓ [1, 5]
✓ [0, 5]
✗ [5]
✓ [0, 4]
✗ [4]
✓ [0, 3]
✗ [3]
✓ [0, 2]
✗ [2]
✓ [0, 1]
✗ [1]
✓ [0, 0]
✗ [0]
✗ [0]

10 shrinks with 17 function calls


In [6]:
def shrink2(ls):
    for i in range(len(ls)):
        s = list(ls)
        del s[i]
        yield list(s)
        
    for i in range(len(ls)):
        for x in range(ls[i]):
            s = list(ls)
            s[i] = x
            yield s

In [7]:
show_trace([5, 5], lambda x: len(x) >= 2, partial(
        greedy_shrink, shrink=shrink2))

✓ [5, 5]
✗ [5]
✗ [5]
✓ [0, 5]
✗ [5]
✗ [0]
✓ [0, 0]
✗ [0]
✗ [0]

2 shrinks with 8 function calls


In [8]:
show_trace([1000], lambda x: sum(x) >= 500,
           partial(greedy_shrink, shrink=shrink2))

✓ [1000]
✗ []
✗ [0]
✗ [1]
✗ [2]
✗ [3]
✗ [4]
✗ [5]
✗ [6]
✗ [7]
✗ [8]
✗ [9]
✗ [10]
✗ [11]
✗ [12]
✗ [13]
✗ [14]
✗ [15]
✗ [16]
✗ [17]
✗ [18]
✗ [19]
✗ [20]
✗ [21]
✗ [22]
✗ [23]
✗ [24]
✗ [25]
✗ [26]
✗ [27]
✗ [28]
✗ [29]
✗ [30]
✗ [31]
✗ [32]
✗ [33]
✗ [34]
✗ [35]
✗ [36]
✗ [37]
✗ [38]
✗ [39]
✗ [40]
✗ [41]
✗ [42]
✗ [43]
✗ [44]
✗ [45]
✗ [46]
✗ [47]
✗ [48]
✗ [49]
✗ [50]
✗ [51]
✗ [52]
✗ [53]
✗ [54]
✗ [55]
✗ [56]
✗ [57]
✗ [58]
✗ [59]
✗ [60]
✗ [61]
✗ [62]
✗ [63]
✗ [64]
✗ [65]
✗ [66]
✗ [67]
✗ [68]
✗ [69]
✗ [70]
✗ [71]
✗ [72]
✗ [73]
✗ [74]
✗ [75]
✗ [76]
✗ [77]
✗ [78]
✗ [79]
✗ [80]
✗ [81]
✗ [82]
✗ [83]
✗ [84]
✗ [85]
✗ [86]
✗ [87]
✗ [88]
✗ [89]
✗ [90]
✗ [91]
✗ [92]
✗ [93]
✗ [94]
✗ [95]
✗ [96]
✗ [97]
✗ [98]
✗ [99]
✗ [100]
✗ [101]
✗ [102]
✗ [103]
✗ [104]
✗ [105]
✗ [106]
✗ [107]
✗ [108]
✗ [109]
✗ [110]
✗ [111]
✗ [112]
✗ [113]
✗ [114]
✗ [115]
✗ [116]
✗ [117]
✗ [118]
✗ [119]
✗ [120]
✗ [121]
✗ [122]
✗ [123]
✗ [124]
✗ [125]
✗ [126]
✗ [127]
✗ [128]
✗ [129]
✗ [130]
✗ [131]
✗ [132]
✗ [133]
✗ [134]
✗ [135]
✗ [136]


In [9]:
def shrink_integer(n):
    if not n:
        return
    for k in range(64):
        probe = 2 ** k
        if probe >= n:
            break
        yield probe - 1
    probe //= 2
    while True:
        probe = (probe + n) // 2
        yield probe
        if probe == n - 1:
            break


def shrink3(ls):
    for i in range(len(ls)):
        s = list(ls)
        del s[i]
        yield list(s)
        for x in shrink_integer(ls[i]):
            s = list(ls)
            s[i] = x
            yield s

In [10]:
show_trace([1000], lambda x: sum(x) >= 500, partial(
        greedy_shrink, shrink=shrink3))

✓ [1000]
✗ []
✗ [0]
✗ [1]
✗ [3]
✗ [7]
✗ [15]
✗ [31]
✗ [63]
✗ [127]
✗ [255]
✓ [511]
✗ []
✗ [0]
✗ [1]
✗ [3]
✗ [7]
✗ [15]
✗ [31]
✗ [63]
✗ [127]
✗ [255]
✗ [383]
✗ [447]
✗ [479]
✗ [495]
✓ [503]
✗ []
✗ [0]
✗ [1]
✗ [3]
✗ [7]
✗ [15]
✗ [31]
✗ [63]
✗ [127]
✗ [255]
✗ [379]
✗ [441]
✗ [472]
✗ [487]
✗ [495]
✗ [499]
✓ [501]
✗ []
✗ [0]
✗ [1]
✗ [3]
✗ [7]
✗ [15]
✗ [31]
✗ [63]
✗ [127]
✗ [255]
✗ [378]
✗ [439]
✗ [470]
✗ [485]
✗ [493]
✗ [497]
✗ [499]
✓ [500]
✗ []
✗ [0]
✗ [1]
✗ [3]
✗ [7]
✗ [15]
✗ [31]
✗ [63]
✗ [127]
✗ [255]
✗ [378]
✗ [439]
✗ [469]
✗ [484]
✗ [492]
✗ [496]
✗ [498]
✗ [499]

4 shrinks with 79 function calls


In [11]:
show_trace([2] * 20, lambda x: sum(x) >= 3, partial(
        greedy_shrink, shrink=shrink3))

✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2, 2]
✓ [2, 2, 2, 2, 2]
✓ [2, 2, 2, 2]
✓ [2, 2, 2]
✓ [2, 2]
✗ [2]
✗ [0, 2]
✓ [1, 2]
✗ [2]
✗ [0, 2]
✗ [1]
✗ [1, 0]
✗ [1, 1]

19 shrinks with 26 function calls


In [12]:
def shrink_to_prefix(ls):
    i = 1
    while i < len(ls):
        yield ls[:i]
        i *= 2


def delete_individual_elements(ls):
    for i in range(len(ls)):
        s = list(ls)
        del s[i]
        yield list(s)


def shrink_individual_elements(ls):
    for i in range(len(ls)):
        for x in shrink_integer(ls[i]):
            s = list(ls)
            s[i] = x
            yield s
            
def shrink4(ls):
    yield from shrink_to_prefix(ls)
    yield from delete_individual_elements(ls)
    yield from shrink_individual_elements(ls)            

In [13]:
show_trace([2] * 20, lambda x: sum(x) >= 3, partial(
        greedy_shrink, shrink=shrink4))

✓ [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
✗ [2]
✓ [2, 2]
✗ [2]
✗ [2]
✗ [2]
✗ [0, 2]
✓ [1, 2]
✗ [1]
✗ [2]
✗ [1]
✗ [0, 2]
✗ [1, 0]
✗ [1, 1]

2 shrinks with 13 function calls


In [14]:
show_trace([20] * 7,
           lambda x: len([t for t in x if t >= 5]) >= 5,
           partial(greedy_shrink, shrink=shrink4))

✓ [20, 20, 20, 20, 20, 20, 20]
✗ [20]
✗ [20, 20]
✗ [20, 20, 20, 20]
✓ [20, 20, 20, 20, 20, 20]
✗ [20]
✗ [20, 20]
✗ [20, 20, 20, 20]
✓ [20, 20, 20, 20, 20]
✗ [20]
✗ [20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [0, 20, 20, 20, 20]
✗ [1, 20, 20, 20, 20]
✗ [3, 20, 20, 20, 20]
✓ [7, 20, 20, 20, 20]
✗ [7]
✗ [7, 20]
✗ [7, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [7, 20, 20, 20]
✗ [7, 20, 20, 20]
✗ [7, 20, 20, 20]
✗ [7, 20, 20, 20]
✗ [0, 20, 20, 20, 20]
✗ [1, 20, 20, 20, 20]
✗ [3, 20, 20, 20, 20]
✓ [5, 20, 20, 20, 20]
✗ [5]
✗ [5, 20]
✗ [5, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [5, 20, 20, 20]
✗ [5, 20, 20, 20]
✗ [5, 20, 20, 20]
✗ [5, 20, 20, 20]
✗ [0, 20, 20, 20, 20]
✗ [1, 20, 20, 20, 20]
✗ [3, 20, 20, 20, 20]
✗ [4, 20, 20, 20, 20]
✗ [5, 0, 20, 20, 20]
✗ [5, 1, 20, 20, 20]
✗ [5, 3, 20, 20, 20]
✓ [5, 7, 20, 20, 20]
✗ [5]
✗ [5, 7]
✗ [5, 7, 20, 20]
✗ [7, 20, 20, 20]
✗ [5, 20, 20, 20]
✗ [5, 7, 20, 20]
✗ [5, 7, 20, 20]
✗ [5, 7, 20, 

In [15]:
def shrink_shared(ls):
    shared_indices = {}
    for i in range(len(ls)):
        shared_indices.setdefault(ls[i], []).append(i)
    for sharing in shared_indices.values():
        if len(sharing) > 1:
            for v in shrink_integer(ls[sharing[0]]):
                s = list(ls)
                for i in sharing:
                    s[i] = v
                yield s


def shrink5(ls):
    yield from shrink_to_prefix(ls)
    yield from delete_individual_elements(ls)
    yield from shrink_shared(ls)
    yield from shrink_individual_elements(ls)

In [16]:
show_trace([20] * 7,
           lambda x: len([t for t in x if t >= 5]) >= 5,
           partial(greedy_shrink, shrink=shrink5))

✓ [20, 20, 20, 20, 20, 20, 20]
✗ [20]
✗ [20, 20]
✗ [20, 20, 20, 20]
✓ [20, 20, 20, 20, 20, 20]
✗ [20]
✗ [20, 20]
✗ [20, 20, 20, 20]
✓ [20, 20, 20, 20, 20]
✗ [20]
✗ [20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [20, 20, 20, 20]
✗ [0, 0, 0, 0, 0]
✗ [1, 1, 1, 1, 1]
✗ [3, 3, 3, 3, 3]
✓ [7, 7, 7, 7, 7]
✗ [7]
✗ [7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [0, 0, 0, 0, 0]
✗ [1, 1, 1, 1, 1]
✗ [3, 3, 3, 3, 3]
✓ [5, 5, 5, 5, 5]
✗ [5]
✗ [5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [0, 0, 0, 0, 0]
✗ [1, 1, 1, 1, 1]
✗ [3, 3, 3, 3, 3]
✗ [4, 4, 4, 4, 4]
✗ [0, 5, 5, 5, 5]
✗ [1, 5, 5, 5, 5]
✗ [3, 5, 5, 5, 5]
✗ [4, 5, 5, 5, 5]
✗ [5, 0, 5, 5, 5]
✗ [5, 1, 5, 5, 5]
✗ [5, 3, 5, 5, 5]
✗ [5, 4, 5, 5, 5]
✗ [5, 5, 0, 5, 5]
✗ [5, 5, 1, 5, 5]
✗ [5, 5, 3, 5, 5]
✗ [5, 5, 4, 5, 5]
✗ [5, 5, 5, 0, 5]
✗ [5, 5, 5, 1, 5]
✗ [5, 5, 5, 3, 5]
✗ [5, 5, 5, 4, 5]

In [17]:
show_trace([20 + i for i in range(7)],
           lambda x: len([t for t in x if t >= 5]) >= 5,
           partial(greedy_shrink, shrink=shrink5))

✓ [20, 21, 22, 23, 24, 25, 26]
✗ [20]
✗ [20, 21]
✗ [20, 21, 22, 23]
✓ [21, 22, 23, 24, 25, 26]
✗ [21]
✗ [21, 22]
✗ [21, 22, 23, 24]
✓ [22, 23, 24, 25, 26]
✗ [22]
✗ [22, 23]
✗ [22, 23, 24, 25]
✗ [23, 24, 25, 26]
✗ [22, 24, 25, 26]
✗ [22, 23, 25, 26]
✗ [22, 23, 24, 26]
✗ [22, 23, 24, 25]
✗ [0, 23, 24, 25, 26]
✗ [1, 23, 24, 25, 26]
✗ [3, 23, 24, 25, 26]
✓ [7, 23, 24, 25, 26]
✗ [7]
✗ [7, 23]
✗ [7, 23, 24, 25]
✗ [23, 24, 25, 26]
✗ [7, 24, 25, 26]
✗ [7, 23, 25, 26]
✗ [7, 23, 24, 26]
✗ [7, 23, 24, 25]
✗ [0, 23, 24, 25, 26]
✗ [1, 23, 24, 25, 26]
✗ [3, 23, 24, 25, 26]
✓ [5, 23, 24, 25, 26]
✗ [5]
✗ [5, 23]
✗ [5, 23, 24, 25]
✗ [23, 24, 25, 26]
✗ [5, 24, 25, 26]
✗ [5, 23, 25, 26]
✗ [5, 23, 24, 26]
✗ [5, 23, 24, 25]
✗ [0, 23, 24, 25, 26]
✗ [1, 23, 24, 25, 26]
✗ [3, 23, 24, 25, 26]
✗ [4, 23, 24, 25, 26]
✗ [5, 0, 24, 25, 26]
✗ [5, 1, 24, 25, 26]
✗ [5, 3, 24, 25, 26]
✓ [5, 7, 24, 25, 26]
✗ [5]
✗ [5, 7]
✗ [5, 7, 24, 25]
✗ [7, 24, 25, 26]
✗ [5, 24, 25, 26]
✗ [5, 7, 25, 26]
✗ [5, 7, 24, 26]
✗ [5, 7, 24, 

In [18]:
def replace_with_simpler(ls):
    if not ls:
        return
    values = set(ls)
    values.remove(max(ls))
    values = sorted(values)
    for v in values:
        yield [min(v, l) for l in ls]


def shrink6(ls):
    yield from shrink_to_prefix(ls)
    yield from delete_individual_elements(ls)
    yield from replace_with_simpler(ls)
    yield from shrink_shared(ls)
    yield from shrink_individual_elements(ls)

In [19]:
show_trace([20 + i for i in range(7)],
           lambda x: len([t for t in x if t >= 5]) >= 5,
           partial(greedy_shrink, shrink=shrink6))

✓ [20, 21, 22, 23, 24, 25, 26]
✗ [20]
✗ [20, 21]
✗ [20, 21, 22, 23]
✓ [21, 22, 23, 24, 25, 26]
✗ [21]
✗ [21, 22]
✗ [21, 22, 23, 24]
✓ [22, 23, 24, 25, 26]
✗ [22]
✗ [22, 23]
✗ [22, 23, 24, 25]
✗ [23, 24, 25, 26]
✗ [22, 24, 25, 26]
✗ [22, 23, 25, 26]
✗ [22, 23, 24, 26]
✗ [22, 23, 24, 25]
✓ [22, 22, 22, 22, 22]
✗ [22]
✗ [22, 22]
✗ [22, 22, 22, 22]
✗ [22, 22, 22, 22]
✗ [22, 22, 22, 22]
✗ [22, 22, 22, 22]
✗ [22, 22, 22, 22]
✗ [22, 22, 22, 22]
✗ [0, 0, 0, 0, 0]
✗ [1, 1, 1, 1, 1]
✗ [3, 3, 3, 3, 3]
✓ [7, 7, 7, 7, 7]
✗ [7]
✗ [7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [7, 7, 7, 7]
✗ [0, 0, 0, 0, 0]
✗ [1, 1, 1, 1, 1]
✗ [3, 3, 3, 3, 3]
✓ [5, 5, 5, 5, 5]
✗ [5]
✗ [5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [5, 5, 5, 5]
✗ [0, 0, 0, 0, 0]
✗ [1, 1, 1, 1, 1]
✗ [3, 3, 3, 3, 3]
✗ [4, 4, 4, 4, 4]
✗ [0, 5, 5, 5, 5]
✗ [1, 5, 5, 5, 5]
✗ [3, 5, 5, 5, 5]
✗ [4, 5, 5, 5, 5]
✗ [5, 0, 5, 5, 5]
✗ [5, 1, 5, 5, 5]
✗ [5, 3, 5, 5, 5]
✗ [5, 

In [20]:
from collections import OrderedDict

conditions = OrderedDict([
    ("length >= 2", lambda xs: len(xs) >= 2),
    ("sum >= 500", lambda xs: sum(xs) >= 500),
    ("sum >= 3", lambda xs: sum(xs) >= 3),
    ("At least 10 by 5", lambda xs: len(
        [t for t in xs if t >= 5]) >= 10),
])

In [21]:
import random

N_EXAMPLES = 100

datasets = {}

def gen_list(rnd):
    return [
        random.getrandbits(64)
        for _ in range(random.randint(0, 100))
    ]

def dataset_for(condition):
    if condition in datasets:
        return datasets[condition]
    constraint = conditions[condition]
    dataset = []
    rnd = random.Random(condition)
    while len(dataset) < N_EXAMPLES:
        ls = gen_list(rnd)
        if constraint(ls):
            dataset.append(ls)
    datasets[condition] = dataset
    return dataset

dataset_for("sum >= 3")[1]

[3793949562060477509,
 9934290619010807846,
 10002020403414014961,
 7711750195787946487,
 9667556573933793070,
 14527157885688042486,
 2955337322900607759,
 5040053509094990192,
 11843429622063020187,
 9625735396817956729,
 10852214825738425134,
 3748440625783927233,
 4659688914107569309,
 14991152893752733926,
 3319294705368929581,
 18262323913697472654,
 12608295579305390098,
 9157257428746368176,
 2210496493814942305,
 14419341534411001417,
 8104656808776167130,
 112011419948734314,
 13375532353579687422,
 17031435954417628514,
 18150064722391718406,
 7789228020387630219,
 8174102717156356645,
 10522370102851365611,
 5379885677883643593,
 7574597871879651223,
 6420461024488265548,
 11942486221511528858,
 3053084725769641661,
 7664362078019878007,
 6298593967394538983,
 16491078252569937825,
 17778971200672453322,
 1936798676827585199,
 16272520037252713923,
 2760374238523886249,
 7211207934820995910,
 8415130166165971412,
 17520369036489258568,
 14718688706977400415,
 97067944910844

In [22]:
MAX_COUNT = 5000

class MaximumCountExceeded(Exception):
    pass

def call_counts(condition, simplifier):
    constraint = conditions[condition]
    dataset = dataset_for(condition)
    counts = []

    for ex in dataset:
        counter = [0]
    
        def run_and_count(ls):
            counter[0] += 1
            if counter[0] > MAX_COUNT:
                raise MaximumCountExceeded()
            return constraint(ls)
    
        try:
            simplifier(ex, run_and_count)
            counts.extend(counter)
        except MaximumCountExceeded:
            counts.append(MAX_COUNT + 1)
    return counts
            
def worst_case(condition, simplifier):
    return max(call_counts(condition, simplifier))

worst_case(
    "length >= 2",
    partial(greedy_shrink, shrink=shrink6))

13

In [23]:
from IPython.display import HTML

def compare_simplifiers(named_simplifiers):
    html_fragments = []
    html_fragments.append("<table>\n<thead>\n<tr>")
    header = ["Condition"]
    header.extend(name for name, _ in named_simplifiers)
    for h in header:
        html_fragments.append("<th>%s</th>" % (h,))
    html_fragments.append("</tr>\n</thead>\n<tbody>")
    
    for name in conditions:
        bits = [name.replace(">", "&gt;")]        
        for _, simplifier in named_simplifiers:
            value = worst_case(name, simplifier)
            if value <= MAX_COUNT:
                bits.append(str(value))
            else:
                bits.append(" &gt; %d" % (MAX_COUNT,))
        html_fragments.append("<tr>  ")
        html_fragments.append(' '.join(
                "<td>%s</td>" % (b,) for b in bits))
        html_fragments.append("</tr>")
    html_fragments.append("</tbody>\n</table>")
    return HTML('\n'.join(html_fragments))

In [24]:
compare_simplifiers([
   (f.__name__[-1], partial(greedy_shrink, shrink=f))
   for f in [shrink2, shrink3, shrink4, shrink5, shrink6]
])

Condition,2,3,4,5,6
length >= 2,106,105,13,13,13
sum >= 500,1102,178,80,80,80
sum >= 3,108,107,9,9,9
At least 10 by 5,534,689,809,877,144


In [25]:
def greedy_shrink_with_dedupe(ls, constraint, shrink):
    seen = set()
    while True:
        for s in shrink(ls):
            key = tuple(s)
            if key in seen:
                continue
            seen.add(key)
            if constraint(s):
                ls = s
                break
        else:
            return ls

In [26]:
compare_simplifiers([
   ("Normal", partial(greedy_shrink, shrink=shrink6)),
   ("Deduped", partial(greedy_shrink_with_dedupe,
                       shrink=shrink6)),

])

Condition,Normal,Deduped
length >= 2,13,6
sum >= 500,80,35
sum >= 3,9,6
At least 10 by 5,144,107


In [27]:
show_trace([100 + i for i in range(10)],
           lambda x: len(set(x)) >= 10,
           partial(greedy_shrink, shrink=shrink6))

✓ [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100]
✗ [100, 101]
✗ [100, 101, 102, 103]
✗ [100, 101, 102, 103, 104, 105, 106, 107]
✗ [101, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100, 101, 103, 104, 105, 106, 107, 108, 109]
✗ [100, 101, 102, 104, 105, 106, 107, 108, 109]
✗ [100, 101, 102, 103, 105, 106, 107, 108, 109]
✗ [100, 101, 102, 103, 104, 106, 107, 108, 109]
✗ [100, 101, 102, 103, 104, 105, 107, 108, 109]
✗ [100, 101, 102, 103, 104, 105, 106, 108, 109]
✗ [100, 101, 102, 103, 104, 105, 106, 107, 109]
✗ [100, 101, 102, 103, 104, 105, 106, 107, 108]
✗ [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
✗ [100, 101, 101, 101, 101, 101, 101, 101, 101, 101]
✗ [100, 101, 102, 102, 102, 102, 102, 102, 102, 102]
✗ [100, 101, 102, 103, 103, 103, 103, 103, 103, 103]
✗ [100, 101, 102, 103, 104, 104, 104, 104, 104, 104]
✗ [100, 101, 102, 103, 104, 105, 105, 105, 105, 105]
✗ [100, 101, 102, 103, 104, 105, 106, 106, 106, 106]
✗ [100, 1

In [28]:
def multicourse_shrink1(ls, constraint):
    seen = set()
    for shrink in [
        shrink_to_prefix,
        replace_with_simpler,
        shrink_shared,
        shrink_individual_elements,
    ]:
        while True:
            for s in shrink(ls):
                key = tuple(s)
                if key in seen:
                    continue
                seen.add(key)
                if constraint(s):
                    ls = s
                    break
            else:
                break
    return ls

In [29]:
show_trace([100 + i for i in range(10)],
           lambda x: len(set(x)) >= 10,
           multicourse_shrink1)

✓ [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100]
✗ [100, 101]
✗ [100, 101, 102, 103]
✗ [100, 101, 102, 103, 104, 105, 106, 107]
✗ [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
✗ [100, 101, 101, 101, 101, 101, 101, 101, 101, 101]
✗ [100, 101, 102, 102, 102, 102, 102, 102, 102, 102]
✗ [100, 101, 102, 103, 103, 103, 103, 103, 103, 103]
✗ [100, 101, 102, 103, 104, 104, 104, 104, 104, 104]
✗ [100, 101, 102, 103, 104, 105, 105, 105, 105, 105]
✗ [100, 101, 102, 103, 104, 105, 106, 106, 106, 106]
✗ [100, 101, 102, 103, 104, 105, 106, 107, 107, 107]
✗ [100, 101, 102, 103, 104, 105, 106, 107, 108, 108]
✓ [0, 101, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [0, 0, 102, 103, 104, 105, 106, 107, 108, 109]
✓ [0, 1, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [0, 1, 0, 103, 104, 105, 106, 107, 108, 109]
✗ [0, 1, 1, 103, 104, 105, 106, 107, 108, 109]
✓ [0, 1, 3, 103, 104, 105, 106, 107, 108, 109]
✗ [0, 0, 3, 103, 104, 105, 106, 107, 108, 109]
✓ [0, 1, 2, 103, 104, 105, 106, 107, 108, 109]

In [30]:
conditions["10 distinct elements"] = lambda xs: len(set(xs)) >= 10

In [31]:
compare_simplifiers([
    ("Single pass", partial(greedy_shrink_with_dedupe,
                            shrink=shrink6)),
    ("Multi pass", multicourse_shrink1)
])

Condition,Single pass,Multi pass
length >= 2,6,4
sum >= 500,35,34
sum >= 3,6,5
At least 10 by 5,107,58
10 distinct elements,623,320


In [32]:
def simplify_index(i):
    def accept(ls):
        if i >= len(ls):
            return
        for v in shrink_integer(ls[i]):
            s = list(ls)
            s[i] = v
            yield s
    return accept

def shrinkers_for(ls):
    yield shrink_to_prefix
    yield delete_individual_elements
    yield replace_with_simpler
    yield shrink_shared
    for i in range(len(ls)):
        yield simplify_index(i)

def multicourse_shrink2(ls, constraint):
    seen = set()
    for shrink in shrinkers_for(ls):
        while True:
            for s in shrink(ls):
                key = tuple(s)
                if key in seen:
                    continue
                seen.add(key)
                if constraint(s):
                    ls = s
                    break
            else:
                break
    return ls

In [33]:
show_trace([100 + i for i in range(10)],
           lambda x: len(set(x)) >= 10,
           multicourse_shrink2)

✓ [100, 101, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100]
✗ [100, 101]
✗ [100, 101, 102, 103]
✗ [100, 101, 102, 103, 104, 105, 106, 107]
✗ [101, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100, 102, 103, 104, 105, 106, 107, 108, 109]
✗ [100, 101, 103, 104, 105, 106, 107, 108, 109]
✗ [100, 101, 102, 104, 105, 106, 107, 108, 109]
✗ [100, 101, 102, 103, 105, 106, 107, 108, 109]
✗ [100, 101, 102, 103, 104, 106, 107, 108, 109]
✗ [100, 101, 102, 103, 104, 105, 107, 108, 109]
✗ [100, 101, 102, 103, 104, 105, 106, 108, 109]
✗ [100, 101, 102, 103, 104, 105, 106, 107, 109]
✗ [100, 101, 102, 103, 104, 105, 106, 107, 108]
✗ [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
✗ [100, 101, 101, 101, 101, 101, 101, 101, 101, 101]
✗ [100, 101, 102, 102, 102, 102, 102, 102, 102, 102]
✗ [100, 101, 102, 103, 103, 103, 103, 103, 103, 103]
✗ [100, 101, 102, 103, 104, 104, 104, 104, 104, 104]
✗ [100, 101, 102, 103, 104, 105, 105, 105, 105, 105]
✗ [100, 101, 102, 103, 104, 105, 106, 106, 106, 106]
✗ [100, 1

In [34]:
show_trace([101, 100],
           lambda x: len(x) >= 2 and x[0] > x[1],
           multicourse_shrink2)

✓ [101, 100]
✗ [101]
✗ [100]
✗ [100, 100]
✗ [0, 100]
✗ [1, 100]
✗ [3, 100]
✗ [7, 100]
✗ [15, 100]
✗ [31, 100]
✗ [63, 100]
✗ [82, 100]
✗ [91, 100]
✗ [96, 100]
✗ [98, 100]
✗ [99, 100]
✓ [101, 0]

1 shrinks with 16 function calls


In [35]:
show_trace([5] * 10,
           lambda x: x and len(x) > min(x),
           multicourse_shrink1)

✓ [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
✗ [5]
✗ [5, 5]
✗ [5, 5, 5, 5]
✓ [5, 5, 5, 5, 5, 5, 5, 5]
✓ [0, 0, 0, 0, 0, 0, 0, 0]

2 shrinks with 5 function calls


In [37]:
conditions["First > Second"] = lambda xs: len(xs) >= 2 and xs[0] > xs[1]

In [38]:
conditions["Size > max & 63"] = lambda xs: xs and len(xs) > (max(xs) & 63)

In [39]:
def multicourse_shrink3(ls, constraint):
    seen = set()
    while True:
        old_ls = ls
        for shrink in shrinkers_for(ls):
            while True:
                for s in shrink(ls):
                    key = tuple(s)
                    if key in seen:
                        continue
                    seen.add(key)
                    if constraint(s):
                        ls = s
                        break
                else:
                    break
        if ls == old_ls:
            return ls

In [40]:
show_trace([101, 100],
           lambda xs: len(xs) >= 2 and xs[0] > xs[1],
           multicourse_shrink3)

✓ [101, 100]
✗ [101]
✗ [100]
✗ [100, 100]
✗ [0, 100]
✗ [1, 100]
✗ [3, 100]
✗ [7, 100]
✗ [15, 100]
✗ [31, 100]
✗ [63, 100]
✗ [82, 100]
✗ [91, 100]
✗ [96, 100]
✗ [98, 100]
✗ [99, 100]
✓ [101, 0]
✗ [0]
✗ [0, 0]
✓ [1, 0]
✗ [1]

2 shrinks with 20 function calls


In [41]:
show_trace([5] * 10,
           lambda x: x and len(x) > min(x),
           multicourse_shrink3)

✓ [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
✗ [5]
✗ [5, 5]
✗ [5, 5, 5, 5]
✓ [5, 5, 5, 5, 5, 5, 5, 5]
✓ [5, 5, 5, 5, 5, 5, 5]
✓ [5, 5, 5, 5, 5, 5]
✗ [5, 5, 5, 5, 5]
✓ [0, 0, 0, 0, 0, 0]
✓ [0]
✗ []

5 shrinks with 10 function calls


In [42]:
compare_simplifiers([
    ("Single pass", partial(greedy_shrink_with_dedupe,
                            shrink=shrink6)),
    ("Multi pass", multicourse_shrink3)
    
])

Condition,Single pass,Multi pass
length >= 2,6,6
sum >= 500,35,35
sum >= 3,6,6
At least 10 by 5,107,73
10 distinct elements,623,131
First > Second,1443,1405
Size > max & 63,387,> 5000


In [52]:
MAX_SHRINKS_PER_RUN = 5


def multicourse_shrink4(ls, constraint):
    seen = set()
    while True:
        old_ls = ls
        shrinks_this_run = 0
        for shrink in shrinkers_for(ls):
            while shrinks_this_run < MAX_SHRINKS_PER_RUN:
                for s in shrink(ls):
                    key = tuple(s)
                    if key in seen:
                        continue
                    seen.add(key)
                    if constraint(s):
                        shrinks_this_run += 1
                        ls = s
                        break
                else:
                    break
        if ls == old_ls:
            return ls

In [53]:
compare_simplifiers([
    ("Single pass", partial(greedy_shrink_with_dedupe,
                            shrink=shrink6)),
    ("Multi pass", multicourse_shrink3),
    ("Multi pass with restart", multicourse_shrink4)    
    
])

Condition,Single pass,Multi pass,Multi pass with restart
length >= 2,6,6,6
sum >= 500,35,35,35
sum >= 3,6,6,6
At least 10 by 5,107,73,77
10 distinct elements,623,131,240
First > Second,1443,1405,1412
Size > max & 63,387,> 5000,1218
Messy,899,> 5000,1798


In [54]:
import hashlib

conditions["Messy"] = lambda xs: hashlib.md5(repr(xs).encode('utf-8')).hexdigest()[0] == '0'

In [55]:
compare_simplifiers([
    ("Single pass", partial(greedy_shrink_with_dedupe,
                            shrink=shrink6)),
    ("Multi pass", multicourse_shrink3),
    ("Multi pass with restart", multicourse_shrink4)    
    
])

Condition,Single pass,Multi pass,Multi pass with restart
length >= 2,6,6,6
sum >= 500,35,35,35
sum >= 3,6,6,6
At least 10 by 5,107,73,77
10 distinct elements,623,131,240
First > Second,1443,1405,1412
Size > max & 63,387,> 5000,1218
Messy,899,> 5000,1798
